## Vincular a drive

In [3]:
import os

In [25]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [5]:
os.listdir()

['.config', 'drive', 'sample_data']

In [ ]:
os.chdir("drive/MyDrive/Aprendizaje/UOC/2023_2024_S5/Git4/TFM")

In [6]:
os.chdir("drive/MyDrive/UOC/S5/TFM")

## Configurar y entrenar GANs

In [7]:
from src.gan import GAN
from src.generator import generator
from src.discriminator import  discriminator

In [9]:
dcgan = GAN(generator =  generator(additional_blocks=3).generator_model,
            discriminator = discriminator(additional_layers=4, depth = 128).discriminator_model,
            path_resultados = "Prueba",
            # data_dir = "Datos/data_npz_64x64_compressed/",
            epochs = 3,
            sample_interval= 1,
            batch_size=128,
            learning_rate=0.0005)

dcgan

Se han cargado 21554 imagenes.
La carpeta 'Prueba' ya existe.


In [ ]:
dcgan.train()

0.0042133331298828125
4/4 [==============================] - 0s 17ms/step - loss: 0.6883 - accuracy: 1.0000
0 [D loss: 0.740227, reals_acc.: 100.00%, false_acc.:100.00%] [G loss: 0.254433 - 1.00%]
32.505643367767334
1/1 [==============================] - 1s 832ms/step
1.3273723125457764
4/4 [==============================] - 0s 17ms/step - loss: 0.6409 - accuracy: 1.0000
1 [D loss: 0.419808, reals_acc.: 99.22%, false_acc.:100.00%] [G loss: 0.063155 - 1.00%]
1.4464054107666016
1/1 [==============================] - 0s 17ms/step
0.290097713470459
4/4 [==============================] - 0s 16ms/step - loss: 0.3249 - accuracy: 1.0000
2 [D loss: 0.320751, reals_acc.: 100.00%, false_acc.:100.00%] [G loss: 1.308977 - 0.04%]
1.3611664772033691
1/1 [==============================] - 0s 17ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Evaluar imagenes

In [19]:
import numpy as np
from src import fid
from keras.applications.inception_v3 import InceptionV3


In [10]:
r, c = 10,15
noise = np.random.normal(0, 1, (r * c, 100))

gen_imgs = dcgan.generator.predict(noise)
gen_imgs = fid.scale_images(gen_imgs, (128, 128, 3))
gen_imgs = fid.preprocess_input(gen_imgs)

5/5 [==============================] - 3s 144ms/step


In [26]:
gen_imgs = fid.generate_images(carpeta = "ID21")


 Resultados/ID21/imagenes FID.npz ya existe.


In [29]:
gen_imgs = fid.scale_images(gen_imgs, (128, 128, 3))
gen_imgs = fid.preprocess_input(gen_imgs)


***Nota:*** En este caso, se parte de un conjunto de imágenes generado anteriormente por el modelo correspondiente al ID21 en la fase final. 

In [30]:
real_data = np.load("Datos/data_npz_64x64_compressed_todas/todas.npz")
real_data = real_data["img"]
real_imgs = fid.sample_real_images(real_data)
real_imgs_scaled = fid.scale_images(real_imgs, (128, 128, 3))
real_imgs_prerpocessed= fid.preprocess_input(real_imgs_scaled)

In [31]:
model = InceptionV3(include_top=False, pooling='avg', input_shape=(128,128,3))

In [32]:
fid_value = fid.calculate_fid(model, real_imgs_prerpocessed, gen_imgs)

313/313 [==============================] - 3s 10ms/step


In [34]:
fid_value

35.86086286646608